## Import modules

In [1]:
import sys
sys.path.append('src')

In [2]:
import pandas
import numpy

from energiapy.components.resource import Resource
from energiapy.components.process import Process
from energiapy.parameters.factor import Factor
from energiapy.components.temporal_scale import TemporalScale
from energiapy.parameters.mpvar import Theta
from energiapy.parameters.bound import BigM
from energiapy.utils.data_utils import make_henry_price_df, remove_outliers
from itertools import product

## Data Import

The following data is needed for the model

- solar and wind profiles : energiapy.fetch_nsrdb_data imports data from the NREL NSRDB database
- power demand : ERCOT for Houston; CAISO for San Diego
- Natural Gas prices: Henry Hub Price Index for both


**Get Weather data**

In [3]:
def load_data(loc:str, index:list):
    df = pandas.read_csv(f'data/{loc}_solar19.csv')
    df['idx'] = index
    df = df.set_index('idx')
    return df

In [4]:
idx = [(i,j,k) for i,j,k in product(range(1), range(365), range(24))]


In [5]:
weather_ny = load_data('ny', idx)[['DNI', 'Wind Speed']]
weather_ny = weather_ny.rename(columns= {'DNI': 'dni', 'Wind Speed': 'wind_speed'})
weather_sd = load_data('sd', idx)[['dni', 'wind_speed']]
weather_ho = load_data('ho', idx)[['dni', 'wind_speed']]


**Demand data for San Diego (CAISO for SDGE region) and Houston (ERCOT for COAST region)**

In [6]:
demand_sd = pandas.read_excel('data/HistoricalEMSHourlyLoad-2019.xlsx', index_col= 0)[['SDGE']]
demand_ho = pandas.read_excel('data/Native_Load_2019.xlsx')[['COAST']]
demand_ny = pandas.DataFrame(pandas.read_csv('data/NYC_load.csv')['Load']) #from Will and Doga

**Natural gas prices from Henry Hub Price Index**  

We use the special function energiapy.make_henry_price_df because of special requirements such as filling in empty data points (weekends, public holidays). Such function will be generalized in the next update of energipy

In [7]:
ng_price = make_henry_price_df(
    file_name='data/Henry_Hub_Natural_Gas_Spot_Price_Daily.csv', year=2020)
ng_price = ng_price.drop(columns= 'scales')
ng_price = remove_outliers(ng_price, sd_cuttoff = 3)
ng_price = pandas.concat([ng_price])

## Define temporal scale


The variabilities of energy systems are best captured over a discretized spatio-temporal scale. In energiapy, the first declaration is the temporal scale. 

For e.g.: Here we declare three temporal scales at different levels from right to left. The interpretation of these scales is merely symentic. Scales can be declared as the problem demands.
- 0, annual, with 1 discretization
- 1, daily with 365 discretization
- 2, hourly with 24 discretization

In essence, we are creating a temporal scale of 8760 points.

In [8]:
horizon = TemporalScale(discretization_list=[5, 365, 24])

## Declare resources

Resources can be consumed, produced, stored, discharged (or sold). 

In [9]:
th_h2 = Theta((0, 10))

In [10]:
Hydrogen = Resource(name='H2', discharge = True, sell_cost = Theta((0, 10)))

In [11]:
Hydrogen.sell_cost.params

[Sell_cost(H2,network,t0)]

In [12]:
Hydrogen.sell_cost

Sell_cost(H2)

In [13]:
from energiapy.parameters.bound import BigM 

In [14]:
cc = Factor(data = pandas.DataFrame({'a': [2]}), scales= horizon, apply_max_scaler= False)


In [15]:
cc

(,t0)

In [16]:
Solar = Resource(name='Solar', gwp =50, eutt = 60, consume=BigM, basis='MW', label='Solar Power')

Wind = Resource(name='Wind', consume=cc, basis='MW', label='Wind Power')

Power = Resource(name='Power', basis='MW', label='Power generated')

Uranium = Resource(name='Ur', consume=pandas.DataFrame({'a': [1, 2, 3, 4, 5]}), scales= horizon, purchase_cost= 42.70/(250/2), basis='kg', label='Uranium')


H2 = Resource(name='H2', basis='tons', label='Hydrogen')

H2_L = Resource(name='H2_L', store= True, basis='tons', label='Hydrogen')

CO2_AQoff = Resource(name='CO2(Aq)', store= True, basis='tons', label='Carbon dioxide - sequestered')

H2O = Resource(name='H2O', consume=(20,50), consume_scale= 2,
               purchase_cost= 0.001, basis='tons', label='Water')

CH4 = Resource(name='CH4', consume=[20, 40], purchase_cost=1, basis='tons', label='Natural gas')

CO2 = Resource(name='CO2', basis='tons', label='Carbon dioxide', block='Resource')

CO2_Vent = Resource(name='CO2_Vent', basis='tons', label='Carbon dioxide - Vented', sell_cost= (2, 20))

O2 = Resource(name='O2', discharge= [20, BigM], basis='tons', label='Oxygen')

CO2_DAC = Resource(name='CO2_DAC', basis='tons', store = {50: pandas.DataFrame({'a': [1, 2, 3, 4, 5]})}, store_loss = 0.01, scales = horizon,  label='Carbon dioxide - captured')


In [17]:
CO2_Vent.eqns()

0<=discharge(CO2_Vent,network,t0)<=BigM
sell_cost(CO2_Vent,network,t0)=Th.discharge(CO2_Vent,network,t0), Th in (2,20)


In [18]:
CH4.eqns()

20<=consume(CH4,network,t0)<=40
purchase_cost(CH4,network,t0)=1.consume(CH4,network,t0)


In [19]:
Uranium.eqns()

0<=consume(Ur,network,t1)<=Consume(Ur,network,t1)
purchase_cost(Ur,network,t0)=0.3416.consume(Ur,network,t0)


In [20]:
H2O.consume.eqns()

consume(H2O,network,t2)<=, Th in (20,50)


In [21]:
H2O.consume.params[0].ub

50

In [22]:
CO2_DAC.store_loss 

Store_loss(CO2_DAC)

In [23]:
from dataclasses import dataclass
from energiapy.utils.data_utils import get_depth

In [24]:
Process.resource_limits()

['STORE', 'CONSUME', 'DISCHARGE', 'CAPACITY']

In [25]:
Process.resource_losses()

['STORE_LOSS']

In [26]:
Process.resource_cashflows()

['SELL_COST', 'PURCHASE_COST', 'STORE_COST', 'CREDIT', 'PENALTY']

## Declare processes

In [27]:
LiI = Process(name='LiI', store= 2000, conversion = {Power: 0.9}, store_loss = 0.2, store_cost= 0.4, store_loss_scale = 2, 
              capex = 3516428, fopex= 87910, vopex = 0, capacity= 1000, label='Lithium-ion battery', basis = 'MW')

# WF = Process(name='WF', conversion={Wind: -1, Power: 1}, capex= (0, 20), fopex= (0, 3), vopex=(0, 1), cap_max= Theta((1, 2)), land = Theta((4, 10)), label='Wind mill array', basis = 'MW')

WF = Process(name='WF', gwp = 50, odp = 2900, conversion={Power: {Wind: -1}},capex=1462000, fopex=43000, vopex=4953, capacity=1000, land_use = 30, label='Wind mill array', basis = 'MW')

# PV = Process(name='PV', conversion={Solar: -1, Power: 1}, capex = 1333262, fopex= 22623, cap_max=1000, label = 'Solar PV', basis = 'MW')
PV = Process(name='PV', scales= horizon, sell_cost= 30, credit = pandas.DataFrame({'a': [2]}), conversion={Power: {1:{Solar: -1.2}, 2: {Solar: -1.5}}}, capex = 1333262, fopex= 22623, capacity=True, label = 'Solar PV', basis = 'MW')

SMRH = Process(name='SMRH', conversion={H2: {Power: -1.11, CH4: -3.76, H2O: -23.7, CO2_Vent: 1.03, CO2: 9.332}}, capex =2520000, fopex = 945000, vopex = 51.5, capacity= 1000, label='Steam methane reforming + CCUS')

NGCC = Process(name= 'NGCC', conversion = {Power: {CH4: -0.108, CO2_Vent: 0.297*0.05, CO2:0.297*0.95}}, capex = 2158928, fopex= 53320, vopex = 4090, capacity= [1,100], label = 'NGCC + 95% CC')

SMR = Process(name='SMR', capex = 2400, fopex = 800, vopex = 0.03,  conversion={H2: {Power: -1.11, CH4: -3.76, H2O: -23.7,CO2_Vent: 9.4979}}, capacity=1000, label='Steam methane reforming')

H2FC = Process(name='H2FC', conversion = {Power: {H2:-0.050}}, capex =  1.6*10**6, vopex = 3.5, capacity = 1000, label = 'hydrogen fuel cell')

DAC = Process(name='DAC', capex = 0.02536, fopex = 0.634, conversion= {CO2_DAC: {Power: -0.193, H2O: -4.048}}, capacity=1000, label='Direct air capture')

PSH = Process(name='PSH', conversion = {Power: 0.6}, capex = 3924781, fopex= 17820, vopex = 512.5, store = 10000, capacity=1000, label='Pumped storage hydropower', basis = 'MW')

ASMR = Process(name='ASMR', consume = {Uranium: 40}, conversion={Power:{Uranium: -4.17*10**(-5), H2O: -3.364}}, capex = 7988951, fopex= 0.04*0.730, capacity = 1000, label='Small modular reactors (SMRs)')

AWE = Process(name='AWE', land_use = (0,20),   conversion= {H2: {Power: -1, O2: 0.7632, H2O: -0.1753}}, capex = 1.1*10**6, fopex = 16918, capacity = BigM, label='Alkaline water electrolysis (AWE)', citation='Demirhan et al. 2018 AIChE paper') 


In [28]:
PV.eqns()

0<=store(Power,LiI,t0)<=2000
0<=capacity(Power,LiI,t0)<=1000
store_cost(Power,LiI,t0)=0.4.store(Power,LiI,t0)
store_loss(Power,LiI,t2)=0.2
capex(Power,PV,process,t0)=1333262.capacity(Power,PV,process,t0)
fopex(Power,PV,process,t0)=22623.capacity(Power,PV,process,t0)


In [29]:
Solar.eqns()

0<=consume(Solar,network,t0)<=BigM
gwp(Solar,network,t0)=50
eutt(Solar,network,t0)=60


In [30]:
Power.eqns()

0<=store(Power,LiI,t0)<=2000
0<=capacity(Power,LiI,t0)<=1000
store_cost(Power,LiI,t0)=0.4.store(Power,LiI,t0)
store_loss(Power,LiI,t2)=0.2
sell_cost(Power,PV,t0)=30.discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0).discharge(Power,PV,t0)


In [31]:
AWE.eqns()

0<=capacity(H2,SMRH,t0)<=1000
capex(H2,AWE,process,t0)=1100000.0.capacity(H2,AWE,process,t0)
fopex(H2,AWE,process,t0)=16918.capacity(H2,AWE,process,t0)
land_use(H2,AWE,process,t0)=Th, Th in (0,20)


In [32]:
Power.eqns()

0<=store(Power,LiI,t0)<=2000
0<=capacity(Power,LiI,t0)<=1000
store_cost(Power,LiI,t0)=0.4.store(Power,LiI,t0)
store_loss(Power,LiI,t2)=0.2
sell_cost(Power,PV,t0)=30.discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0).discharge(Power,PV,t0)


In [33]:
AWE.eqns()

0<=capacity(H2,SMRH,t0)<=1000
capex(H2,AWE,process,t0)=1100000.0.capacity(H2,AWE,process,t0)
fopex(H2,AWE,process,t0)=16918.capacity(H2,AWE,process,t0)
land_use(H2,AWE,process,t0)=Th, Th in (0,20)


In [34]:
H2.eqns()

0<=capacity(H2,SMRH,t0)<=1000


In [35]:
Uranium.consume.eqns()

0<=consume(Ur,network,t1)<=Consume(Ur,network,t1)
0<=consume(Ur,ASMR,t0)<=40


In [36]:
Uranium.eqns()

0<=consume(Ur,network,t1)<=Consume(Ur,network,t1)
purchase_cost(Ur,network,t0)=0.3416.consume(Ur,network,t0)


In [37]:
NGCC.eqns()

0<=store(Power,LiI,t0)<=2000
0<=capacity(Power,LiI,t0)<=1000
store_cost(Power,LiI,t0)=0.4.store(Power,LiI,t0)
store_loss(Power,LiI,t2)=0.2
sell_cost(Power,PV,t0)=30.discharge(Power,PV,t0)
credit(Power,PV,t0)=Credit(Power,PV,t0).discharge(Power,PV,t0)
capex(Power,NGCC,process,t0)=2158928.capacity(Power,NGCC,process,t0)
fopex(Power,NGCC,process,t0)=53320.capacity(Power,NGCC,process,t0)
vopex(Power,NGCC,process,t0)=4090.discharge(Power,NGCC,process,t0)


AttributeError: 'dict' object has no attribute 'eqns'

In [ ]:
process_list = {LiI, WF, PV, SMRH, SMR, DAC, AWE, H2_L_c, H2_L_d, AQoff_SMR, ASMR, PSH, NGCC}

## Declare locations

In [ ]:
cc = Factor(data = pandas.DataFrame({'a': [2]}), scales= scales, apply_max_scaler= False)

houston = Location(name='HO', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, credit = {PV: 2}, cap_max_localize = {WF: 0.4, AWE: 23}, 
                   credit_factor ={PV: pandas.DataFrame({'a': [2]})}, consume_localize = {Solar: 2}, demand= {Power: (0, 1)}, 
                   land_cost = 10, land_cost_factor= cc, capacity_factor={PV: weather_ho[['dni']], WF: weather_ho[['wind_speed']]}, demand_factor={
                   Power: demand_ho}, discharge_factor={Power: demand_ho}, purchase_cost_factor={CH4: ng_price}, land_max= True, scales=scales, label='Houston')

sandiego = Location(name='SD', processes={LiI, PV, WF, SMRH, AWE, AQoff_SMR}, purchase_cost_localize = {CH4: 222}, 
                    consume_localize = {Solar: 3}, land_cost = 30, land_cost_factor = pandas.DataFrame({'a': [2]}) , 
                    credit = {SMRH: 30, AQoff_SMR: Theta((0, 23))}, credit_factor= {SMRH: pandas.DataFrame({'a': [1 - 0.001*i for i in range(365)]})}, 
                    capacity_factor={PV: weather_sd[['dni']], WF: weather_sd[['wind_speed']]}, demand_factor={
                    Power: demand_sd}, purchase_cost_factor={CH4: ng_price}, discharge_localize= {O2: 0.2}, scales=scales, label='SanDiego')

newyork = Location(name='NY', processes={LiI, H2FC}, land_max = (0, 50000), land_max_factor = pandas.DataFrame({'a': [2]}), cap_max_localize = {PV: 0.2, AWE: 34}, 
                   capacity_factor={PV: weather_ny[['dni']], WF: weather_ny[['wind_speed']]}, demand= {Power: 40}, demand_factor={
                   Power: demand_ny}, credit = {PV: 7}, purchase_cost_localize = {CH4: 2}, scales=scales, label='NewYork')


In [ ]:
PV.cap_max

In [ ]:
PV.factors

In [ ]:
houston.ptype

In [ ]:
houston.land_cost_factor

In [ ]:
# for i in houston.processes | sandiego.processes | newyork.processes:
# # for i in houston.resources | sandiego.resources | newyork.resources:
#     print(i, i.localizations)
#     print(i, i.factors)
    

In [ ]:
Power.ctype

## Declare transports

In [ ]:

# Train_H2 = Transport(name='Train_H2', resources={
#                      H2}, trans_max=bigM, trans_loss=0.03, trans_cost=1.667*10**(-3), label='Railway for hydrogen transportation')
# Grid = Transport(name='Grid', resources={
#                  Power}, trans_max=bigM, trans_loss=0.001, trans_cost=0.5*10**(-3), label='Railroad transport')


Train_H2 = Transport(name='Train_H2', resources={
                     H2}, cap_max=BigM, trans_loss=0.03, capex = 1000, vopex=1.667, fopex = (0,1), acid = 34, eutf =55, label='Railway for hydrogen transportation')
Grid = Transport(name='Grid', resources={
                 H2, Power}, cap_max=BigM, trans_loss=0.001, capex = 500, vopex = 0.444, label='Railroad transport')



In [ ]:
Train_H2.cap_max

## Declare Network

In [ ]:
distance_matrix = [
    [2366], 
    [2620]
]

transport_matrix = [
    [[Train_H2]], 
    [[Grid]],
]
capacity_factor  = {(houston, newyork): {Train_H2: weather_sd[['wind_speed']]}, (sandiego, newyork): {Train_H2: weather_sd[['wind_speed']]}}

network = Network(name='network', scales = scales, land_max= (0,5), land_max_factor= pandas.DataFrame({'a': [5]}),  
                  capacity_factor= capacity_factor, vopex_factor = {(houston, sandiego): {Train_H2: weather_sd[['wind_speed']]}}, sources=[houston, sandiego],
                  sinks=[newyork], distance_matrix=distance_matrix, transport_matrix=transport_matrix)


In [ ]:

scenario = Scenario(name='scenario_full', network=network, scales=scales, label='full_case')


In [ ]:
scenario

In [ ]:
for i in houston.resources:
    print(i.ctype)

In [ ]:
houston.resources_produce

In [ ]:
ResourceType.all()

In [ ]:
ResourceType.resource_level()

In [ ]:
ResourceType.location_level()

In [ ]:
ResourceType.transport_level()

In [ ]:
ProcessType.all()

In [ ]:
ProcessType.process_level()

In [ ]:
ProcessType.location_level()

In [ ]:
LocationType.location_level()

In [ ]:
Power.transport

In [ ]:
Solar.availability

In [ ]:
Power.demand

In [ ]:
[((houston, sandiego), CO2), ((houston, sandiego), H2), ((houston, newyork), CO2)]

In [ ]:
comp_loc_dict = scenario.resources_locations_discharge

In [ ]:
scenario.locations_land

In [ ]:
scenario.locations_locations_land

In [ ]:
comp_loc_ordered_set = [(i, j) for i in comp_loc_dict for j in comp_loc_dict[i]]

In [ ]:
scenario.resources_locations_store

In [ ]:
comp_loc_ordered_set

In [ ]:
comp_loc_ordered_set

In [ ]:
scenario.resources_locations_discharge

In [ ]:
scenario.resources_discharge

In [ ]:
scenario.processes_locations_expenditure

In [ ]:
resources_consume_locations = {resource: set() for resource in scenario.resources_consume}
for resource in scenario.resources_consume:
    for location in scenario.locations:
        if hasattr(location, 'resources_consume'):
            if resource in location.resources_consume:
                resources_consume_locations[resource].add(location)

In [ ]:
resources_consume_locations = {resource: {location for location in scenario.locations if hasattr(location, 'resources_consume') and resource in location.resources_consume} for resource in scenario.resources_consume}

In [ ]:
scenario.resources_consume

In [ ]:
resources_consume_locations = {resource: {location for location in self.locations if hasattr(location, 'resources_consume') and resource in location.resources_consume} for resource in self.resources_consume}

In [ ]:
resources_consume_locations 

In [ ]:
resources_demand_locations = {resource: {location for location in scenario.locations if hasattr(location, 'resources_demand') and resource in location.resources_demand} for resource in scenario.resources_demand}

In [ ]:
scenario.resource_classifications()

In [ ]:
{component: {location for location in getattr(self, 'locations') if hasattr(location, f'{component_set}_{parameter.lower()}') and resource in getattr(location, f'{component_set}_{parameter.lower()}')} for resource in getattr(self, f'{component_set}_{parameter.lower()}')}

In [ ]:
scenario.process_classifications()


In [ ]:
[((houston.name, sandiego.name), CO2.name), ((houston.name, sandiego.name), H2.name), ((houston.name, newyork.name), CO2.name)]

In [ ]:
help(ResourceParamType)

In [ ]:
from pyomo.environ import *
# from energiapy.utils.scale_utils import scale_pyomo_set


In [ ]:
scales1 = TemporalScale(discretization_list=[1, 4], design_scale= 0, scheduling_scale=1)

In [ ]:
type(scales1)

In [ ]:
scales2 = TemporalScale(discretization_list=[2, 2, 2], design_scale= 0, scheduling_scale=2)


In [ ]:
scales1.index_list

In [ ]:
scales2.index_list

In [ ]:
m = ConcreteModel()
m.scales = Set(scales1.list,
                    initialize=scales1.scale)
m.scales_scheduling = scale_pyomo_set(
        instance=m, scale_level=scales1.scheduling_scale)
m.scales_design = scale_pyomo_set(
        instance=m, scale_level=scales1.design_scale)



In [ ]:
scales.scheduling_scale

In [ ]:
# m.a = Set(initialize = [(i, j) for i in resources_consume_locations for j in resources_consume_locations[i]])
# m.b = Set(initialize = [(i.name, j.name) for i in scenario.resources_locations_consume for j in scenario.resources_locations_consume[i]])
# m.c = Set(initialize = [((houston.name, sandiego.name), CO2.name), ((houston.name, sandiego.name), H2.name), ((houston.name, newyork.name), CO2.name)])
# m.A = Var(m.a, m.scales_scheduling, within = NonNegativeReals)
# m.B = Var(m.b, m.scales_design, within = NonNegativeReals)
# m.C = Var(m.c, m.scales_scheduling, within = NonNegativeReals)

In [ ]:
from pyomo.environ import * 
m = ConcreteModel()

In [ ]:
m.d = Set(initialize = [(Train_H2.name, (houston.name, sandiego.name)), (Grid.name, (houston.name, sandiego.name)), (Grid.name, (houston.name, newyork.name))])
m.D = Var(m.d, m.scales_design, within = NonNegativeReals)


In [ ]:
m.f = Set(initialize = [Train_H2.name, Grid.name])
m.F = Var(m.f, m.scales_design, within = NonNegativeReals)


In [ ]:
m.g = Set(initialize = [(CO2.name, (houston.name, sandiego.name)), (Power.name, (houston.name, sandiego.name)), (H2.name, (houston.name, newyork.name))])

In [ ]:
m.G = Var(m.g, m.scales_scheduling, within = NonNegativeReals)

In [ ]:
m.G.pprint()

In [ ]:
m = ConcreteModel()


In [ ]:
import gurobipy
m = gurobipy.read('example.lp')

In [ ]:
import pandas 

In [ ]:
a = pandas.DataFrame(data= {'a': [1, 2, 3]})

In [ ]:
b = 

In [ ]:
class LPFileCreator:
    def __init__(self, objective, constraints, bounds, file_name="problem.lp"):
        self.objective = objective
        self.constraints = constraints
        self.bounds = bounds
        self.file_name = file_name
    
    def create_lp_file(self):
        content = self._generate_lp_content()
        with open(self.file_name, "w") as file:
            file.write(content)
    
    def _generate_lp_content(self):
        lp_content = []
        
        # Add Objective Function
        lp_content.append("\\* Problem Statement *\\")
        lp_content.append("\nMaximize")
        lp_content.append(f" obj: {self.objective}\n")
        
        # Add Constraints
        lp_content.append("Subject To")
        for i, constraint in enumerate(self.constraints, start=1):
            lp_content.append(f" c{i}: {constraint}")
        lp_content.append("")
        
        # Add Bounds
        lp_content.append("Bounds")
        for bound in self.bounds:
            lp_content.append(f" {bound}")
        lp_content.append("")
        
        # Add End
        lp_content.append("End")
        
        return "\n".join(lp_content)


In [ ]:
a = [7, 2, 3].sort()


In [ ]:
a

In [ ]:
sorted(a)

In [ ]:
houston.resources_consume

In [ ]:
[f'cons_{i.name}_' for i in houston.resources_consume]

In [ ]:
def tuples_to_strings(tuple_list):
    return ['_'.join(map(str, t)) for t in tuple_list]

In [ ]:
scales = TemporalScale([1,4], design_scale= 0, scheduling_scale=1)

In [ ]:
var = ['cons'] #, 'sell']

In [ ]:
res_list = ['res1', 'res2', 'res3']

In [ ]:
time_index = tuples_to_strings(scales.index_list[1])

In [ ]:
value = {'res1': 5, 'res2': 7, 'res3': 8}

In [ ]:
cons = list()

In [ ]:
for i,j,k in product(var, res_list, time_index):
    cons.append(f'{i}_{j}_{k} <= {value[j]}')

In [ ]:
ooo = {'res1': 7, 'res2': 5, 'res3': 9}

In [ ]:
k = [f'{ooo[i]} {i}' for i in res_list]

In [ ]:
obj = ' + '.join(k)
obj

In [ ]:
for i,j,k in product(var, res_list, time_index):


In [ ]:
bou = [f'0 <= {i}' for i in var]

In [ ]:

# Define the problem components
objective = "x + 2 y"
constraints = [
    "x + y >= 1",
    "x <= 2",
    "y <= 2"
]
bounds = [
    "0 <= x",
    "0 <= y"
]

# Create the LP file
# creator = LPFileCreator(objective, constraints, bounds, file_name="example.lp")

creator = LPFileCreator(obj, cons, bou, file_name="example.lp")
creator.create_lp_file()
print("LP file 'example.lp' created successfully.")

In [ ]:
import gurobipy as gb 

a = gb.read('example.lp')

In [ ]:
a.optimize()

In [ ]:
Maximize
  x + y + z
Subject To
  c0: x + y = 1
  c1: x + 5 y + 2 z <= 10
  qc0: x + y + [ x ^ 2 - 2 x * y + 3 y ^ 2 ] <= 5
Bounds
  0 <= x <= 5
  z >= 2
Generals
  x y z
End


In [ ]:
H2O.purchase_cost

In [ ]:
H2O.cons_max

In [ ]:
m.optimize()

In [ ]:
m.getVars()

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= houston, process= PV, fig_size= (9,5), color= 'orange')


![cap fac pv](plots/cf_ho.png)

In [ ]:
plot_scenario.capacity_factor(scenario = scenario, location= sandiego, process= WF, fig_size= (9,5), color= 'blue')


![cap fac wf](plots/wf_sd.png)

In [ ]:
plot_scenario.demand_factor(scenario = scenario, location= newyork, resource= Power, fig_size= (9,5), color= 'red')


![df ny](plots/df_ny.png)

## Optimize for minimum cost

In [ ]:
milp_cost = formulate(scenario= scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION, Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.COST)

**Ensure no discharge of power in Houston and San Diego**

While the demand for power is zero in these locations, Power is still a dischargeable resource. Hence, the discharge bounds need to be set to zero.
If not set to zero, the system chooses to dispense power instead of establishing storage networks.

In [ ]:

milp_cost.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_cost.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_cost, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_cost = solve(scenario = scenario, instance= milp_cost, solver= 'gurobi', name="results_cost", print_solversteps = True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_cost, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny](plots/sch_sdny.png)

In [ ]:
plot_results.transport(results=results_cost, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch hony](plots/sch_hony.png)

**Cost contribution by technology**

In [ ]:
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.CAPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.VOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.FOPEX, location= 'HO', fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.PROCESS_WISE, y = CostY.TOTAL, location= 'HO', fig_size= (8,6))

![total cost ho](plots/total_ho.png)

**Location-wise cost contribution by technology**

In [ ]:

plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.CAPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.VOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.FOPEX, fig_size= (8,6))
plot_results.cost(results= results_cost, x = CostX.LOCATION_WISE, y = CostY.TOTAL, fig_size= (8,6))

![total lw](plots/total_lw.png)

**Inventory and production schedules**

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'Inv', component= 'LiI_Power_stored', location = 'SD', fig_size= (9,5), color = 'steelblue')

![inv lii](plots/lii_inv.png)

In [ ]:
plot_results.schedule(results= results_cost, y_axis= 'P', component= 'PV', location = 'SD', fig_size= (9,5), color = 'steelblue')

![pv p](plots/pv_p.png)

## Optimize to maximize resource discharge 

In [ ]:

milp_demand = formulate(scenario=scenario, constraints={Constraints.COST, Constraints.INVENTORY, Constraints.PRODUCTION,
                        Constraints.RESOURCE_BALANCE, Constraints.TRANSPORT, Constraints.MODE, Constraints.NETWORK},  objective=Objective.MAX_DISCHARGE, objective_resource=Power)


In [ ]:

milp_demand.constraint_specific_location_discharge_ho = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = houston)
milp_demand.constraint_specific_location_discharge_sd = constraint_specific_location_discharge(
        instance=milp_demand, network_scale_level=0, bounds={Power: 0}, location = sandiego)


In [ ]:

results_demand = solve(scenario=scenario, instance=milp_demand, solver='gurobi',
                       name="results_demand", print_solversteps=True)


**Schedule for transport of hydrogen**

In [ ]:
plot_results.transport(results=results_demand, source='SD', sink='NY', resource='H2', transport='Train_H2')


![sch t sdny2](plots/sch_sdny2.png)

In [ ]:
plot_results.transport(results=results_demand, source='HO', sink='NY', resource='H2', transport='Train_H2')

![sch t hony](plots/sch_hony2.png)